In [ ]:
# %%
##DATa ingestion
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader('https://docs.langchain.com/langsmith/administration-overview')
docs=loader.load()
docs



# %%
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_spitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

final_docs=text_spitter.split_documents(docs)

# %%
document1=final_docs[0]
document1

# %%
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# %%
embeddings = OllamaEmbeddings(model="nomic-embed-text")
db=FAISS.from_documents(final_docs,embeddings)
db

# %%
q="LangSmith has two usage limits: total traces are extended"
res=db.similarity_search(q)
res

# %%
q="what is langsmith"
res=db.similarity_search_with_score(q)
res

# %%
##Retreival chain, document chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

from langchain_community.llms import Ollama

llm = Ollama(
    model="llama2"
)

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question besed only on provided context:
<context>
{context}
<context>
"""
)

# document chain :
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

# %%
from langchain_core.documents import Document
document_chain.invoke({
    "input": "LangSmith has two usage limits: total traces and extended",
    "context": [Document(page_content="LangSmith has two usage limits: total traces are extended traces.")]
})

# %%
##Retreiver : it is an interface its respnsibitity to get the data from vectorstoredb 

db

# %%
retreiver=db.as_retriever()
from langchain_classic.chains import create_retrieval_chain
retreival_chain=create_retrieval_chain(retreiver,document_chain)


# %%
retreival_chain

# %%
##get the response from the llm
res=retreival_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
res

# %%



